In [2]:
import sys, os, time

In [3]:
sys.path.append('lib')

In [4]:
import autoreview

In [5]:
config = autoreview.Config()

In [6]:
MAG_PAPER_DATA = '/home/jporteno/data/MAG_academicgraphdls_20180329/parquet/mag_20180329_Papers_withEFandCl_parquet/'
MAG_CITATIONS_DATA = '/home/jporteno/data/MAG_academicgraphdls_20180329/parquet/mag_20180329_PaperReferences_out_parquet/'

In [63]:
MAG_PAPER_DATA = '../../data/MAG_academicgraphdls_20180329/parquet/mag_20180329_Papers_withEFandCl_parquet/'
MAG_CITATIONS_DATA = '../../data/MAG_academicgraphdls_20180329/parquet/mag_20180329_PaperReferences_out_parquet/'

In [64]:
spark = config.spark

In [65]:
sdf_papers = spark.read.parquet(MAG_PAPER_DATA)

In [9]:
sdf_papers

DataFrame[Paper_ID: bigint, mag_rank: int, DOI: string, doc_type: string, title: string, original_title: string, book_title: string, year: int, date: date, publisher: string, Journal_ID: bigint, Conference_series_ID: bigint, Conference_instance_ID: bigint, volume: string, issue: string, first_page: string, last_page: string, reference_count: bigint, citation_count: bigint, estimated_citation: int, cl: string, toplevel: int, EF: float]

In [12]:
x = sdf_papers.groupby('toplevel').count().toPandas()

In [36]:
len(x)

1902250

In [37]:
x.sort_values('count', ascending=False).iloc[1000:1020]

,toplevel,count
532433,1021.0,7739
1464233,1023.0,7736
1121232,1058.0,7727
570119,1069.0,7704
703556,1048.0,7703
1435410,1027.0,7701
9658,1025.0,7701
1626092,1086.0,7692
1320765,1083.0,7692
1158877,1007.0,7683


In [38]:
cl_subset = sdf_papers[sdf_papers['toplevel']==1048]
df_cl_subset = cl_subset.toPandas()

In [39]:
df_cl_subset.head()

,Paper_ID,mag_rank,DOI,doc_type,title,original_title,book_title,year,date,publisher,...,volume,issue,first_page,last_page,reference_count,citation_count,estimated_citation,cl,toplevel,EF
0,202630324,23722,None,None,feature selection in linear models,Feature Selection in Linear Models,None,2008,2008-01-01,None,...,None,None,None,None,5,0,0,1048:4:79,1048,5.050350e-09
1,577139289,20454,None,None,a probabilistic model for dirty multi task fea...,A Probabilistic Model for Dirty Multi-task Fea...,None,2015,2015-01-01,International Machine Learning Society (IMLS),...,None,None,1073,1082,29,4,4,1048:59:25,1048,5.492450e-09
2,1708200757,22088,None,None,sparse empirical bayes analysis seba,Sparse Empirical Bayes Analysis (SEBA),None,2009,2009-11-30,None,...,None,None,None,None,13,0,0,1048:5:226,1048,5.050350e-09
3,1710695479,22332,10.1007/978-3-319-10593-2_38,None,efficient sparsity estimation via marginal las...,Efficient Sparsity Estimation via Marginal-Las...,None,2014,2014-09-06,"Springer, Cham",...,None,None,578,592,32,0,0,1048:9:205,1048,5.050350e-09
4,1769664844,19815,None,None,graph structured multi task regression and an ...,Graph-Structured Multi-task Regression and an ...,None,2010,2010-01-01,None,...,None,None,None,None,19,38,120,1048:25:15,1048,1.298440e-08


In [40]:
sdf_citations = spark.read.parquet(MAG_CITATIONS_DATA)

In [41]:
sdf_citations

DataFrame[Paper_ID: bigint, Paper_Reference_ID: bigint]

In [42]:
sdf_citations = sdf_citations.withColumnRenamed('Paper_ID', 'src')
sdf_citations = sdf_citations.withColumnRenamed('Paper_Reference_ID', 'dst')
cond = [sdf_citations['src']==cl_subset['Paper_ID'], sdf_citations['dst']==cl_subset['Paper_ID']]
sdf_citations_sample = sdf_citations.join(cl_subset, on=cond[0], how='inner').select('src', 'dst')
sdf_citations_sample = sdf_citations_sample.join(cl_subset, on=cond[1], how='inner').select('src', 'dst')

In [43]:
sdf_citations_sample

DataFrame[src: bigint, dst: bigint]

In [44]:
sdf_citations_sample.count()

85247

In [45]:
sdf_citations.count()

1213134880

In [46]:
cl_subset.write.parquet('/home/jporteno/code/autoreview_testpublic/autoreview/sample_data/MAG_papers_sample')

In [47]:
sdf_citations_sample = sdf_citations_sample.withColumnRenamed('src', 'Paper_ID')
sdf_citations_sample = sdf_citations_sample.withColumnRenamed('dst', 'Paper_Reference_ID')

In [48]:
sdf_citations_sample.write.parquet('/home/jporteno/code/autoreview_testpublic/autoreview/sample_data/MAG_citations_sample')

In [49]:
cl_subset

DataFrame[Paper_ID: bigint, mag_rank: int, DOI: string, doc_type: string, title: string, original_title: string, book_title: string, year: int, date: date, publisher: string, Journal_ID: bigint, Conference_series_ID: bigint, Conference_instance_ID: bigint, volume: string, issue: string, first_page: string, last_page: string, reference_count: bigint, citation_count: bigint, estimated_citation: int, cl: string, toplevel: int, EF: float]

In [57]:
paper_ids = df_cl_subset.sample(n=50, random_state=999)['Paper_ID'].tolist()

In [59]:
with open('sample_data/sample_IDs_MAG.txt', 'w') as outf:
    for paper_id in paper_ids:
        outf.write('{}\n'.format(paper_id))